Import Required Libraries and Load Configuration Files

In [2]:
# # Import required libraries  
import os  
import base64
from pathlib import Path
from shutil import rmtree
from requests import get, post
import json
import time
import copy 
from datetime import datetime, timedelta  
import pdfkit
from langchain.text_splitter import TokenTextSplitter, MarkdownHeaderTextSplitter


from openai import AzureOpenAI
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [3]:
#Load the configuration details for the Cognitive Search Service and Azure OpenAI Instance
#Credentials should be secured using a more secure method such as Azure KeyVault
config = json.load(open("config.json"))

Create Azure OpenAI and Document Intelligent (Pre-Built Layout) Instances. Set Chunking Config to split on Markdown Headers

In [4]:
#Azure OpenAI
api_base = config["openai_api_base"]
api_key = config["openai_api_key"]
openai_api_version = config["openai_api_version"]
embeddings_model = config["openai_embedding_model"]
gpt_model = config["openai_gpt_model"] 

# Doc Intelligence Config
di_endpoint = config["doc_intelligence_endpoint"]
di_apim_key = config["doc_intelligence_apim_key"]
di_headers = {
    'Content-Type': 'application/pdf',
    'Ocp-Apim-Subscription-Key': di_apim_key,
}
di_post_url = di_endpoint + "documentintelligence/documentModels/prebuilt-layout:analyze?api-version=2023-10-31-preview&stringIndexType=utf16CodeUnit&outputContentFormat=markdown"
credential = DefaultAzureCredential()

# Chunking Config
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=52)  
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)


# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    api_version=openai_api_version,
    azure_endpoint=api_base,
    api_key=api_key)

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=di_endpoint,
    credential=credential
)

Load Files in 'Data' Folder, Test Layout Analysis on ISDA-2.pdf

In [5]:
data_root_dir = config["data_root_dir"]

# Get all files in dir
def get_files_in_dir(in_dir):
    return [os.path.join(dp, f) for dp, dn, filenames in os.walk(in_dir) for f in filenames]

files = get_files_in_dir(data_root_dir)

In [6]:
for local_file in files:
    
    file_type = os.path.splitext(local_file)[1].lower()
    if file_type != ".pdf":
        continue

    if local_file != "data\lch_ltd_formatted_template-v2.pdf":
        continue
    
    else:
        print("Analyzing %s" % local_file)
        
    with open(local_file, "rb") as f:
        data_bytes = f.read()
    
    
        resp = post(url = di_post_url, data = data_bytes, headers = di_headers)
        if resp.status_code != 202:
            print("POST analyze failed:\n%s" % resp.text)
            quit()
        print("POST analyze succeeded:\n%s" % resp.headers)
        get_url = resp.headers["operation-location"]
        print (get_url)
            
        n_tries = 10
        n_try = 0
        wait_sec = 2
        processing = True
        while processing:
            try:
                resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": di_apim_key})
                resp_json = json.loads(resp.text)
                if resp.status_code != 200:
                    print("GET Layout results failed:\n")
                    processing = False
                elif resp_json["status"] == "succeeded":
                    print("Layout Analysis succeeded:\n")
                    print("--------------------------------")
                    processing = False
                elif resp_json["status"] == "failed":
                    # print("Layout Analysis failed:\n%s" % resp_json)
                    print("Layout Analysis failed:\n")
                    processing = False
                else:
                    # Analysis still running. Wait and retry.
                    print ('Waiting to complete processing...')
                    time.sleep(wait_sec)
            except Exception as e:
                msg = "GET analyze results failed:\n%s" % str(e)
                print(msg)
                processing = False
            
    # Persist the Doc Int Output for further processing
    if 'analyzeResult' in resp_json:
        resp = resp_json['analyzeResult']
    
        break
    


Analyzing data\lch_ltd_formatted_template-v2.pdf
POST analyze succeeded:
{'Content-Length': '0', 'Operation-Location': 'https://klau-docintel.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-layout/analyzeResults/387955ca-4c2c-4d46-8888-df81bad47873?api-version=2023-10-31-preview', 'x-envoy-upstream-service-time': '105', 'apim-request-id': '387955ca-4c2c-4d46-8888-df81bad47873', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'East US', 'Date': 'Mon, 10 Mar 2025 17:06:32 GMT'}
https://klau-docintel.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-layout/analyzeResults/387955ca-4c2c-4d46-8888-df81bad47873?api-version=2023-10-31-preview
Waiting to complete processing...
Waiting to complete processing...
Layout Analysis succeeded:

--------------------------------


Print Tables Found

In [7]:
for table_idx, table in enumerate(resp["tables"]):
    print(
        "Table # {} has {} rows and {} columns".format(
        table_idx, table["rowCount"], table["columnCount"]
        )
    )
        
    for cell in table["cells"]:
        print(
            "...Cell[{}][{}] has content '{}'".format(
            cell["rowIndex"],
            cell["columnIndex"],
            cell["content"]),
            )
        

Table # 0 has 51 rows and 4 columns
...Cell[0][0] has content 'Country'
...Cell[0][1] has content 'Government Securities (Description & Bloomberg Code)'
...Cell[0][2] has content 'Term'
...Cell[0][3] has content 'Haircut (Conventional Bonds)'
...Cell[1][0] has content 'Australia'
...Cell[1][1] has content 'Australian Treasury Bill'
...Cell[1][2] has content '5 business days <= 2yr'
...Cell[1][3] has content '0.78%'
...Cell[2][1] has content '(ACTB)'
...Cell[2][2] has content '> 2yr <= 5yrs'
...Cell[2][3] has content '1.23%'
...Cell[3][1] has content 'Australian Government Bond'
...Cell[3][2] has content '> 5yrs <= 7 yrs'
...Cell[3][3] has content '2.50%'
...Cell[4][1] has content '(ACGB)'
...Cell[4][2] has content '>7yrs <= 10yrs'
...Cell[4][3] has content '3.75%'
...Cell[5][1] has content 'Australian Government I/L'
...Cell[5][2] has content '>10yrs <= 40yrs'
...Cell[5][3] has content '9.25%'
...Cell[6][1] has content '(ACGB)'
...Cell[6][2] has content '>40yrs <= 60yrs'
...Cell[6][3] 

Print Raw Markdown

In [8]:
print(resp["content"])


| Country | Government Securities (Description & Bloomberg Code) | Term | Haircut (Conventional Bonds) |
| - | - | - | - |
| Australia | Australian Treasury Bill | 5 business days <= 2yr | 0.78% |
|| (ACTB) | > 2yr <= 5yrs | 1.23% |
|| Australian Government Bond | > 5yrs <= 7 yrs | 2.50% |
|| (ACGB) | >7yrs <= 10yrs | 3.75% |
|| Australian Government I/L | >10yrs <= 40yrs | 9.25% |
|| (ACGB) | >40yrs <= 60yrs | 10.35% |
| Austria | Austrian Treasury Bill | 18 business days <= 1yr | 0.45% |
|| (RATB) | >1yr <= 4yrs | 1.75% |
|| Austrian Government Bond | >4yrs <= 8 yrs | 2.85% |
|| (RAGB) | >8yrs <= 12yrs | 3.25% |
|| Austria International Bond (AUST) 
||| || || |
| Belgium | Belgian Treasury Bill (BGTB) | 6 business days <= 1yr | 0.89% |
|| Belgian Government Bond | >1yr <= 2yrs | 1.96% |
|| (BGB) | > 2yrs <= 5 yrs | 3.65% |
|| | > 5yrs <= 8yrs | 5.68% |
|| | > 8yrs <= 17yrs | 8.00% |
|| | > 17yrs <= 55yrs | 13.75% |
| Canada | Canadian Treasury Bill (CTB) | 15 calendar days <= 2yr | 

Use Structued Outputs with Azure OpenAI. Define the class basemodel and properties (ex: asset name, party type, tenure max and min years)

In [9]:
from typing import Optional
from pydantic import BaseModel

In [10]:
class Collateral(BaseModel):
    name: Optional[str]
    tenure_min: Optional[str]
    tenure_max: Optional[str]
    haircut_percentage: Optional[str]

class Invoice(BaseModel):
    collaterals: Optional[list[Collateral]]

In [11]:
if resp["tables"]:
    completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": """You are an AI assistant that specializes in extracting collateral information from a single, large table that spans multiple pages. 
            The table contains Government Securities from multiple countries along with different Maturity levels and Haircut values. Each page is a continuation of the previous one, and all must be combined into a **single unified table**. 
            
            ### Extraction Instructions: 
            1. **Multi-Page Table Handling**: \ 
            - Ensure **all pages are processed** together as part of the same table. 
            - **Do not stop after the first page**—continue extracting all consecutive pages. 
            
            2. **Columns to Extract**:
            Combine the collateral Table with different countries' Government Securities and different Maturity levels spans multiple pages, all of which must be extracted for each asset value. Make sure you get all the consecutive pages from australia to italy to luxembourg to UK agencies etc, as every page is part of the one table. 
            - **Government Securities (Asset Type)**: Extract into multiple rows if different asset types are present. Extract as many asset types that are present. 
            - **Tenure Min Maturity** is part of the Term column. Anything on the LEFT side of the sign (<) OR more than sign (>), is the value WITH the sign. Example: 2yr <= Maturity <= 5 yrs would return '2yr <='. Another example: 5yrs <= Maturity < 7 yrs would return '5yrs <='. Note that the values can repeat. 
            - **Tenure Max Maturity**: is part of the Term column. Anything on the RIGHT side of the less than sign (<), is the value AND the sign. Example: Maturity < 9M would return '< 9M'. 2.5Y < Maturity <= 5.5Y would return '<= 5.5Y'. Note that the values can repeat. 
            - **The Haircut Percentage**: is in its own column. 

            3. **Data Structuring**:
            - **Extract each government security as a separate row** if multiple asset types are listed.
            - If a country has multiple securities, **list them individually** rather than merging.
            - Ensure the **tenure values are consistently extracted**, repeating where necessary.

            Make sure the extraction continues through **all countries, from Australia to Italy to Luxembourg to UK agencies, etc.**, ensuring that the entire dataset is fully included in the output."""
        },

        {
            "role": "user",
            "content": resp["content"],
        }
    ],
    response_format=Invoice,
    max_tokens=16384,
    temperature=0.1,
    top_p=0.1
)

In [12]:
#event = completion.choices[0].message.parsed
#print(event.model_dump_json(indent=2))

print("Total token usage: ")
print(completion.usage)


Total token usage: 
CompletionUsage(completion_tokens=5408, prompt_tokens=4490, total_tokens=9898, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


In [13]:
event = completion.choices[0].message.parsed

print(event.model_dump_json(indent=2))

{
  "collaterals": [
    {
      "name": "Australian Treasury Bill",
      "tenure_min": "5 business days <=",
      "tenure_max": "<= 2yr",
      "haircut_percentage": "0.78%"
    },
    {
      "name": "Australian Treasury Bill",
      "tenure_min": "> 2yr <=",
      "tenure_max": "<= 5yrs",
      "haircut_percentage": "1.23%"
    },
    {
      "name": "Australian Government Bond",
      "tenure_min": "> 5yrs <=",
      "tenure_max": "<= 7 yrs",
      "haircut_percentage": "2.50%"
    },
    {
      "name": "Australian Government Bond",
      "tenure_min": ">7yrs <=",
      "tenure_max": "<= 10yrs",
      "haircut_percentage": "3.75%"
    },
    {
      "name": "Australian Government I/L",
      "tenure_min": ">10yrs <=",
      "tenure_max": "<= 40yrs",
      "haircut_percentage": "9.25%"
    },
    {
      "name": "Australian Government I/L",
      "tenure_min": ">40yrs <=",
      "tenure_max": "<= 60yrs",
      "haircut_percentage": "10.35%"
    },
    {
      "name": "Austrian Tr